# WhisperX Cloud API Server 🚀 (Universal)

**支持平台:** Google Colab | Kaggle | 本地 GPU 服务器

这个 Notebook 自动检测运行环境并部署 WhisperX API 服务。

## 🎯 特性
- ✅ 自动检测环境 (Colab/Kaggle/Local)
- ✅ 自动 GPU/CPU 检测
- ✅ **Mamba 包管理**（比 Conda 快 3-5x）
- ✅ **预编译 WhisperX Wheel** 支持（加速安装）
- ✅ 内置 ngrok 隧道（公共 URL）
- ✅ 支持多语言
- ✅ 单词级时间戳对齐
- ✅ 可选说话人分离

## 📋 使用步骤
1. **设置 GPU**: 在 Colab/Kaggle 中启用 GPU
2. **配置 ngrok**: 在下面设置 ngrok token
3. **运行所有单元格**: 点击 Runtime → Run all
4. **复制 URL**: 将生成的公共 URL 复制到 VideoLingo 配置

---

## 0️⃣ 准备项目文件

如果是 Colab/Kaggle 环境，需要先克隆项目文件。本地运行可跳过此步骤。

In [ ]:
import os
import sys

# 检测环境
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = os.path.exists('/kaggle')

# 设置持久化目录
if IN_COLAB:
    PERSIST_DIR = '/content/videolingo-whisperx'
elif IN_KAGGLE:
    PERSIST_DIR = '/kaggle/working/videolingo-whisperx'
else:
    PERSIST_DIR = '.'

REPO_URL = 'https://github.com/infinite-gaming-studio/VideoLingo.git'

if IN_COLAB or IN_KAGGLE:
    # 创建持久化目录
    os.makedirs(PERSIST_DIR, exist_ok=True)
    
    # 检查是否已有仓库
    repo_dir = os.path.join(PERSIST_DIR, 'VideoLingo')
    
    if os.path.exists(os.path.join(repo_dir, '.git')):
        print("🔄 更新项目代码...")
        !cd {repo_dir} && git pull
    else:
        print("📥 克隆项目到持久化目录...")
        !git clone {REPO_URL} {repo_dir}
    
    # 复制文件到当前工作目录
    !cp -r {repo_dir}/whisperx_cloud/* .
    print(f"✅ 项目文件已同步到: {PERSIST_DIR}")
else:
    print("✅ 本地环境: 使用当前目录文件")

## 1️⃣ 环境配置

In [ ]:
%run config.py

## 2️⃣ 环境检测与设置

In [ ]:
%run step1_detect_env.py

## 3️⃣ GPU 检查

In [ ]:
%run step2_check_gpu.py

## 4️⃣ 安装 Mamba 和依赖

自动安装 Miniforge/Mamba（如需要）并创建环境。

**Mamba vs Conda:**
- Mamba 是更快的包管理器（3-5x 速度提升）
- 自动优先使用 Mamba，回退到 Conda

**国际 CDN 加速（自动）:**
- 自动使用多个国际 CDN 加速 WhisperX git 安装
- 支持的 CDN: ghps.cc, ghproxy.net, github.moeyy.xyz 等
- 无需手动配置，自动重试可用 CDN

**自定义 CDN（可选）:**
- 可设置 `WHISPERX_GIT_URL` 自定义 git 源

In [ ]:
# 可选：自定义 WhisperX git 源（默认使用 CDN 加速）
# import os
# os.environ['WHISPERX_GIT_URL'] = 'git+https://your-fast-mirror.com/m-bain/whisperx.git@7307306'

# 可选：设置 HuggingFace 镜像（中国大陆）
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

%run step3_install_deps.py

## 5️⃣ 检查 API 服务器

API 服务器代码位于 `whisperx_server.py`，已预先配置好。

In [ ]:
import os
if os.path.exists('whisperx_server.py'):
    print('✅ Server file found: whisperx_server.py')
else:
    print('❌ Server file not found!')

## 6️⃣ 配置 ngrok

自动使用 mamba/conda 环境的 Python

In [ ]:
%run step4_run_with_conda.py step5_setup_ngrok.py

## 7️⃣ 启动服务器

自动使用 mamba/conda 环境的 Python

In [ ]:
%run step4_run_with_conda.py step6_start_server.py

## 8️⃣ 测试 API

自动使用 mamba/conda 环境的 Python

In [ ]:
%run step4_run_with_conda.py step7_test_api.py

## 9️⃣ 保持运行 (⚠️ 不要停止此单元格!)

自动使用 mamba/conda 环境的 Python

In [ ]:
%run step4_run_with_conda.py step8_keep_running.py

## 📚 VideoLingo 配置指南

服务器运行后，将 URL 配置到 VideoLingo:

### 方法 1: 修改 config.yaml
```yaml
whisper:
  runtime: 'cloud'
  whisperX_cloud_url: 'https://xxxx.ngrok-free.app'  # 你的 ngrok URL
```

### 方法 2: 环境变量
```bash
export WHISPERX_CLOUD_URL='https://xxxx.ngrok-free.app'
```

### 方法 3: VideoLingo 客户端
使用 `whisperx_cloud_client.py` 中的 `WhisperXCloudClient` 类

## 🔧 故障排除

### 1. ngrok 连接失败
- 检查 token 是否正确
- 确认网络连接（Kaggle 需要 Internet ON）
- 尝试重新运行第 5、6 单元格

### 2. GPU 未检测到
- Colab: Runtime → Change runtime type → GPU
- Kaggle: Settings → Accelerator → GPU T4 x2

### 3. 模型下载慢
- 设置 HF_ENDPOINT 为镜像站（中国大陆）
- 或等待首次下载完成（约 3GB）

### 4. 显存不足
- 使用较小模型: 'medium' 或 'small'
- 减小 batch_size
- 禁用 speaker_diarization

### 5. 依赖安装慢/失败
- 安装过程会自动使用 Mamba（比 Conda 快 3-5x）
- 如 Mamba 安装失败，会自动回退到 Conda
- 可使用预编译 WhisperX wheel 加速（见步骤 4）

### 6. WhisperX 安装失败
- 默认会尝试从 git 安装
- 如需更快安装，使用预编译 wheel（步骤 4）
- 或手动构建: `pip wheel git+https://github.com/m-bain/whisperx.git@7307306`